Stippling Sandbox

In [8]:
from PIL import Image, ImageDraw, ImageFilter
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from rasterize import Scanline_Rasterize_Polygon, Sort_Vertices, Bounding_Box, Raster_Centroid, Raster_BBox

In [9]:
def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)


def Outline_Poly(polygon,color,width):
    for i, point in enumerate(polygon):

        k = i + 1
        if k == len(polygon):
            k = 0
        point2 = polygon[k]

        draw.line((point[0],point[1],point2[0],point2[1]),fill=color, width=width)

Image parameters

In [10]:
image_resolution = 1080
image_path = 'sphere.jpg'
weight_image = Image.open(image_path)
weight_image = weight_image.filter(ImageFilter.BoxBlur(5))
res = image_resolution
jitter_mag = 5

scale_factor = image_resolution / res
# img = Image.new('RGB', (image_resolution, image_resolution))
img = Image.open(image_path)
img = weight_image

In [11]:
imageArray = np.array(weight_image)
def Weighted_Raster_Centroid(pixels):
    cx = 0
    cy = 0
    total_weight = 0
    for pixel in pixels:
        sample_x = int(np.floor(pixel[1]))
        sample_y = int(np.round(pixel[0]))

        sampled_value = imageArray[sample_x][sample_y]

        weight = 1 - np.mean(sampled_value) / 255

        if weight < 0.05:
            weight = 0

        total_weight += weight
        
        cx += pixel[0] * weight
        cy += pixel[1] * weight
    
    if total_weight == 0:
        return False

    cx /= total_weight
    cy /= total_weight
    
    return [cx, cy]

In [12]:
def Lloyd_Relax(seeds, res, weight_image):
    vor = Voronoi(seeds)

    polygons = []
    open_cells = []
    for index, region in enumerate(vor.regions):
        poly = []
        if len(region) == 0:
            continue
        elif -1 in region:
            open_cells.append(index)
            continue
        for index in region:
            if index != -1:
                poly.append(vor.vertices[index])
            else:
                print("OUTSIDE")
        
        sorted_poly = Sort_Vertices(poly)

        polygons.append(sorted_poly)

    bbox = [[0,0],[image_resolution,image_resolution]]
    centroids = []
    for polygon in polygons:

        bbox = Raster_BBox(polygon, image_resolution)
        rastered_polygon = Scanline_Rasterize_Polygon(polygon, bbox, image_resolution)
        # centroid = Raster_Centroid(rastered_polygon)
        centroid = Weighted_Raster_Centroid(rastered_polygon)
        if centroid:
            if centroid[0] < 0:
                centroid[0] = 0
            elif centroid[0] > image_resolution:
                centroid[0] = image_resolution
            if centroid[1] < 0:
                centroid[1] = 0
            elif centroid[1] > image_resolution:
                centroid[1] = image_resolution

            centroids.append(centroid)
    
    # new_seeds = centroids
    new_seeds = centroids + open_cells

    # for seed in new_seeds:
    #     x_jitter = (np.random.rand() - 0.5) * jitter_mag
    #     y_jitter = (np.random.rand() - 0.5) * jitter_mag
    #     seed = [seed[0] + x_jitter, seed[1] + y_jitter]
    print(len(open_cells))
    return new_seeds

Creating seeds

In [13]:
seeds = []
startseeds = seeds
point_count = 250

## Rejection sampling
low_thresh = 60
mid_thresh = 125
high_thresh = 225

n = 0
while n < point_count:
    rand_x = np.random.rand() * image_resolution
    rand_y = np.random.rand() * image_resolution
    sampled_value = weight_image.getpixel((rand_x,rand_y))
    if np.mean(sampled_value) < 225:
        seeds.append(np.array([rand_x,rand_y]))
        n += 1
        
## Seed image
seed_img = Image.open(image_path)
draw = ImageDraw.Draw(seed_img)
for seed in startseeds:
    center_ellipse(seed[0], seed[1],2,'red')

seed_img.save("seed_image.jpg")

In [14]:
img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)
for seed in seeds:
    center_ellipse(seed[0], seed[1],2,'white')
    img.save(f'sequence/stipple_result_{0}.png')
    
for i in range(3):
    new_seeds = Lloyd_Relax(seeds,res, weight_image)

    seeds = new_seeds
    print(f"Iteration {i}")
    img = Image.new('RGB', (image_resolution, image_resolution))
    draw = ImageDraw.Draw(img)
    for seed in seeds:
        center_ellipse(seed[0], seed[1],2,'white')
        img.save(f'sequence/stipple_result_{i + 1}.png')




14
Iteration 0


TypeError: 'int' object is not subscriptable